[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/weaviate/recipes/blob/main/integrations/llm-agent-frameworks/mem0/quickstart_mem0_with_weaviate.ipynb)

# Parallel and Weaviate

This notebook will illustrate how to create an AI system with DSPy that uses Parallel's Chat with the Web API and Weaviate's Query Agent!

### Chat with the Web with Parallel 

In [ ]:
import os
from openai import OpenAI

parallel_client = OpenAI(
    api_key=os.environ["PARALLEL_API_KEY"],  # Your Parallel API key
    base_url="https://api.parallel.ai"  # Parallel's API beta endpoint
)

def get_parallel_response(
    parallel_client: OpenAI,
    query: str
) -> str:
    response = parallel_client.chat.completions.create(
        model="speed", # Parallel model name
        messages=[
            {"role": "user", "content": query}
        ]
    )
    return response.choices[0].message.content

response = get_parallel_response(parallel_client, "What does Parallel Web Systems do?")
print(response)

Parallel Web Systems is a technology company that focuses on adapting the web for artificial intelligences. It builds web data and APIs for AI agents, and its work spans innovations across crawling, indexing, ranking, retrieval, and reasoning systems. The company's initial product is a set of APIs for AIs to do more with web data, turning web search and knowledge tasks into programmable, enterprise-ready infrastructure. Parallel aims to build a new programmatic web specifically for AIs with declarative, composable layers built around reasoning and computation, verifiable provenance, and open markets. ([parallel.ai](https://parallel.ai/), [linkedin.com](https://www.linkedin.com/company/parallel-web), [crunchbase.com](https://www.crunchbase.com/organization/parallel-463d), [aws.amazon.com](https://aws.amazon.com/marketplace/seller-profile?id=seller-rqcgpu6w7coya), [builtinsf.com](https://www.builtinsf.com/company/parallel-web-systems))

Parallel's APIs are priced per request and not per 

### Chat with Data Stored in Weaviate with Weaviate's Query Agent

In [25]:
import weaviate
from weaviate.classes.init import Auth
from weaviate.agents.query import QueryAgent

weaviate_client = weaviate.connect_to_weaviate_cloud(
    cluster_url=os.getenv("WEAVIATE_URL"),
    auth_credentials=Auth.api_key(os.getenv("WEAVIATE_API_KEY"))
)

query_agent = QueryAgent(
    client=weaviate_client,
    collections=["IRPapersText_Default"]
)

response = query_agent.ask(
    "How does HyDE work?"
)

response.display()

╭──────────────────────────── 💬 Ask Mode Response ────────────────────────────╮
│                                                                              │
│ HyDE (Hypothetical Document Embeddings) is a method designed for zero-shot   │
│ dense retrieval that does not require any supervised relevance labels. It    │
│ works by decomposing the retrieval task into two main components:            │
│                                                                              │
│ 1. **Generative Hypothetical Document Creation:**                            │
│    Given a user query, an instruction-following large language model (LLM)   │
│ such as InstructGPT is prompted to generate a hypothetical document that     │
│ "answers" the query. This generated text is not a real document and may      │
│ contain hallucinated or factually inaccurate information, but it is intended │
│ to capture the relevance patterns of what a relevant document for that query │
│ might look like.           

### Combine Retrieval from Parallel and Weaviate with DSPy

In [18]:
import dspy

lm = dspy.LM(
    "openai/gpt-4.1"
)

dspy.configure(lm=lm)

lm("say hello")

['Hello! How can I help you today? 😊']

In [ ]:
class GenerateAnswer(dspy.Signature):
    """Assess the context and answer the question."""

    web_context = dspy.InputField(description="Context gathered from the web.")
    my_knowledge_base_context = dspy.InputField(description="Context gathered from my knowledge base.")
    question = dspy.InputField(description="The question to answer.")

    answer = dspy.OutputField(description="The answer to the question.")

class QASystem(dspy.Module):
    def __init__(
        self,
        parallel_client: OpenAI,
        weaviate_client: weaviate.client.WeaviateClient,
        weaviate_collections: list[str]
    ):
        self.parallel_client = parallel_client
        self.weaviate_client = weaviate_client
        self.weaviate_query_agent = QueryAgent(
            client=self.weaviate_client,
            collections=weaviate_collections
        )

        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)
    
    def _get_parallel_response(
        self,
        query: str
    ) -> str:
        response = self.parallel_client.chat.completions.create(
            model="speed", # Parallel model name
            messages=[
                {"role": "user", "content": query}
            ]
        )
        return response.choices[0].message.content
    
    def _get_weaviate_query_agent_response(
        self,
        query: str
    ) -> str:
        return self.weaviate_query_agent.ask(query).final_answer
    
    def __call__(
        self,
        question: str
    ) -> str:
        web_context = self._get_parallel_response(question)
        my_knowledge_base_context = self._get_weaviate_query_agent_response(question)

        answer = self.generate_answer(
            web_context=web_context,
            my_knowledge_base_context=my_knowledge_base_context,
            question=question
        )

        return answer
    
qasystem = QASystem(
    parallel_client=parallel_client,
    weaviate_client=weaviate_client,
    weaviate_collections=["IRPapersText_Default"]
)

response = qasystem("What is HyDE and how many citations does it have?")

print(response.answer)

HyDE (Hypothetical Document Embeddings) is a zero-shot dense retrieval method that improves information retrieval by first prompting a large language model to generate a hypothetical document answering a query, then encoding this document into an embedding vector using a contrastive encoder. This embedding is used to retrieve similar real documents from a corpus, enabling effective retrieval without the need for relevance labels or supervised training.

The original HyDE paper, "Precise Zero-Shot Dense Retrieval without Relevance Labels," has 578 citations as of November 2025.
